In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [3]:
traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

#traindata=traindata.drop('id',axis=1)
#testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])
traindata['attack_cat'] = lb_make.fit_transform(traindata['attack_cat'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])
testdata['attack_cat'] = lb_make.fit_transform(testdata['attack_cat'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-2]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-2]

In [4]:
Arr = X.values
arr = x.values

import librosa
Arr_Delta = librosa.feature.delta(Arr)
arr_delta = librosa.feature.delta(arr)

Delta1 = pd.DataFrame(Arr_Delta)
delta1 = pd.DataFrame(arr_delta)

XDelta1 = pd.concat([X, Delta1], axis=1)
xdelta1 = pd.concat([x, delta1], axis=1)

scaler = Normalizer().fit(XDelta1)
XDelta1 = scaler.transform(XDelta1)
scaler = Normalizer().fit(xdelta1)
xdelta1 = scaler.transform(xdelta1)

Y = np.array(Y)
y = np.array(y)

XDelta1 = np.reshape(XDelta1, (XDelta1.shape[0],XDelta1.shape[1],1))
xdelta1 = np.reshape(xdelta1, (xdelta1.shape[0],xdelta1.shape[1],1))

lstm_output_size = 70

Y = to_categorical(Y)
y = to_categorical(y)

In [7]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(82, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation="softmax"))
cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
cnn.summary()

/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(82, 1), padding="same")`
  
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 82, 64)            256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 41, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2624)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               336000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 337,546
Trainable params: 337,546
Non-trainable params: 0
________________________________________________

In [8]:
history = cnn.fit(XDelta1, Y, epochs=200, validation_data=(xdelta1, y))


Train on 175341 samples, validate on 82332 samples
Epoch 1/200
175341/175341 [==============================] - 119s 677us/step - loss: 1.3927 - accuracy: 0.5303 - val_loss: 1.2809 - val_accuracy: 0.6178
Epoch 2/200
175341/175341 [==============================] - 114s 648us/step - loss: 1.3471 - accuracy: 0.5354 - val_loss: 1.2767 - val_accuracy: 0.6186
Epoch 3/200
175341/175341 [==============================] - 114s 649us/step - loss: 1.3350 - accuracy: 0.5405 - val_loss: 1.2874 - val_accuracy: 0.5959
Epoch 4/200
175341/175341 [==============================] - 121s 690us/step - loss: 1.3296 - accuracy: 0.5432 - val_loss: 1.2720 - val_accuracy: 0.5994
Epoch 5/200
175341/175341 [==============================] - 133s 759us/step - loss: 1.3251 - accuracy: 0.5436 - val_loss: 1.2829 - val_accuracy: 0.5924
Epoch 6/200
175341/175341 [==============================] - 141s 806us/step - loss: 1.3230 - accuracy: 0.5453 - val_loss: 1.2720 - val_accuracy: 0.5979
Epoch 7/200
175341/175341 [===

175341/175341 [==============================] - 143s 815us/step - loss: 1.3066 - accuracy: 0.5510 - val_loss: 1.2684 - val_accuracy: 0.5851
Epoch 53/200
175341/175341 [==============================] - 145s 825us/step - loss: 1.3066 - accuracy: 0.5509 - val_loss: 1.2737 - val_accuracy: 0.5829
Epoch 54/200
175341/175341 [==============================] - 144s 820us/step - loss: 1.3064 - accuracy: 0.5520 - val_loss: 1.2768 - val_accuracy: 0.5852
Epoch 55/200
175341/175341 [==============================] - 145s 825us/step - loss: 1.3061 - accuracy: 0.5512 - val_loss: 1.2815 - val_accuracy: 0.5836
Epoch 56/200
175341/175341 [==============================] - 145s 826us/step - loss: 1.3058 - accuracy: 0.5517 - val_loss: 1.2824 - val_accuracy: 0.5859
Epoch 57/200
175341/175341 [==============================] - 144s 821us/step - loss: 1.3060 - accuracy: 0.5515 - val_loss: 1.2664 - val_accuracy: 0.5902
Epoch 58/200
175341/175341 [==============================] - 145s 826us/step - loss: 1.3

175341/175341 [==============================] - 82s 468us/step - loss: 1.2918 - accuracy: 0.5512 - val_loss: 1.2766 - val_accuracy: 0.5852
Epoch 106/200
175341/175341 [==============================] - 82s 470us/step - loss: 1.2916 - accuracy: 0.5503 - val_loss: 1.2617 - val_accuracy: 0.5856
Epoch 107/200
175341/175341 [==============================] - 82s 469us/step - loss: 1.2921 - accuracy: 0.5499 - val_loss: 1.2817 - val_accuracy: 0.5789
Epoch 108/200
175341/175341 [==============================] - 82s 468us/step - loss: 1.2915 - accuracy: 0.5512 - val_loss: 1.2549 - val_accuracy: 0.5884
Epoch 109/200
175341/175341 [==============================] - 82s 470us/step - loss: 1.2921 - accuracy: 0.5501 - val_loss: 1.2894 - val_accuracy: 0.5541
Epoch 110/200
175341/175341 [==============================] - 83s 471us/step - loss: 1.2920 - accuracy: 0.5498 - val_loss: 1.2698 - val_accuracy: 0.5645
Epoch 111/200
175341/175341 [==============================] - 82s 470us/step - loss: 1.29

175341/175341 [==============================] - 82s 469us/step - loss: 1.2881 - accuracy: 0.5503 - val_loss: 1.2707 - val_accuracy: 0.5833
Epoch 159/200
175341/175341 [==============================] - 82s 470us/step - loss: 1.2897 - accuracy: 0.5504 - val_loss: 1.2829 - val_accuracy: 0.5708
Epoch 160/200
175341/175341 [==============================] - 82s 469us/step - loss: 1.2902 - accuracy: 0.5496 - val_loss: 1.3319 - val_accuracy: 0.5457
Epoch 161/200
175341/175341 [==============================] - 82s 469us/step - loss: 1.2921 - accuracy: 0.5491 - val_loss: 1.2923 - val_accuracy: 0.5414
Epoch 162/200
175341/175341 [==============================] - 82s 470us/step - loss: 1.2902 - accuracy: 0.5504 - val_loss: 1.2656 - val_accuracy: 0.5845
Epoch 163/200
175341/175341 [==============================] - 83s 471us/step - loss: 1.2889 - accuracy: 0.5502 - val_loss: 1.2750 - val_accuracy: 0.5866
Epoch 164/200
175341/175341 [==============================] - 82s 469us/step - loss: 1.28

In [11]:
from sklearn.metrics import accuracy_score
predictions = cnn.predict([xdelta1])

import sys
np.set_printoptions(threshold=sys.maxsize)
#pred = np.zeros((82332,1), dtype=int)
for i in range(0, predictions.shape[0]):
    pred = np.where(predictions[i] == np.amax(predictions[i]))
    for j in range(0,10):
        predictions[i,j] = 0
    predictions[i,pred]=1
    
print(predictions[:5])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [12]:
print("Accuracy: ",accuracy_score(y,predictions))
from sklearn.metrics import multilabel_confusion_matrix,classification_report,confusion_matrix
print("Classification Report: \n",classification_report(y,predictions))
conmat = np.array(confusion_matrix(y.argmax(axis=1), predictions.argmax(axis=1)))
confusion = pd.DataFrame(conmat, index=['0','1','2','3','4','5','6','7','8','9'],
                        columns=[['0','1','2','3','4','5','6','7','8','9']])
print (conmat)
conmat.shape

Accuracy:  0.5863455278627994


/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       677
           1       1.00      0.00      0.01       583
           2       0.95      0.00      0.01      4089
           3       0.24      0.35      0.29     11132
           4       0.34      0.01      0.01      6062
           5       0.58      0.96      0.73     18871
           6       0.75      0.70      0.73     37000
           7       0.00      0.00      0.00      3496
           8       0.00      0.00      0.00       378
           9       0.00      0.00      0.00        44

   micro avg       0.59      0.59      0.59     82332
   macro avg       0.39      0.20      0.18     82332
weighted avg       0.58      0.59      0.53     82332
 samples avg       0.59      0.59      0.59     82332

[[    0     0     0    45     0   602    30     0     0     0]
 [    0     2     0    64     3   485    29     0     0     0]
 [    0     0    19   819    29  258

(10, 10)